In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [2]:
model = tf.keras.models.load_model('final_simple_v1.0')
labels = {
        0 : "0", 1 : "1", 2 : "2", 3 : "3", 4 : "4", 5 : "5",
        6 : "6", 7 : "7", 8 : "8", 9 : "9", 10 : "A", 11 : "B",
        12 : "C c", 13 : "D", 14 : "E", 15 : "F", 16 : "G", 17 : "H",
        18 : "I i", 19 : "J j", 20 : "K k", 21 : "L l", 22 : "M m",
        23 : "N", 24 : "O o", 25 : "P p", 26 : "Q", 27 : "R", 28 : "S s",
        29 : "T", 30 : "U u", 31 : "V v", 32 : "W w", 33 : "X x", 34 : "Y y",
        35 : "Z z", 36 : "a", 37 : "b", 38 : "d", 39 : "e", 40 : "f",
        41 : "g", 42 : "h", 43 : "n", 44 : "q", 45 : "r", 46 : "t"}

In [3]:
import kivy
from io import BytesIO
import os
import tensorflow as tf
import numpy as np
from kivy.app import App
from kivy.uix.button import Button
from kivy.uix.floatlayout import FloatLayout
from kivy.config import Config
from kivy.uix.widget import Widget
from kivy.graphics import Line
from kivy.uix.label import Label
from PIL import Image

[INFO   ] [Logger      ] Record log in C:\Users\renat\.kivy\logs\kivy_22-01-20_6.txt
[INFO   ] [deps        ] Successfully imported "kivy_deps.angle" 0.3.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.glew" 0.3.0
[INFO   ] [deps        ] Successfully imported "kivy_deps.sdl2" 0.3.1
[INFO   ] [Kivy        ] v2.0.0
[INFO   ] [Kivy        ] Installed at "c:\users\renat\appdata\local\programs\python\python39\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.9.6 (tags/v3.9.6:db3ff76, Jun 28 2021, 15:26:21) [MSC v.1929 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "c:\users\renat\appdata\local\programs\python\python39\python.exe"
[INFO   ] [Factory     ] 186 symbols loaded
[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succe

In [4]:
Config.set('graphics', 'resizable', True)
Config.set('graphics', 'width', 500) 
Config.set('graphics', 'height', 350)

Config.set('graphics', 'fullscreen', 0)
Config.set('graphics', 'window_state', 'visible')
Config.set('graphics', 'show_cursor', 1)

In [5]:
def formatOutput( arr):
    prediction = model.predict( arr)[0]
    indexes = np.argsort( prediction)
    
    result = "\n\n\n\n"
    for i in range(1, 6):
        index = indexes[-i]
        result += labels[index] + ": {:.2f}%\n".format( prediction[index]*100)
    return result

def getImageNpArray( kivyImg):
    # treba vidjet kak se odma iz kivy image pretvori u np array
    # ovo je lose rjesenje
    kivyImg.save( "image.png")
    pilImage = Image.open( "image.png", "r", ["png"])
    pilImage = pilImage.resize( (28,28), Image.NEAREST)
    os.remove( "image.png")
    
    array = np.array( pilImage)
    array = array[:,:, 0]
    return array.reshape( (-1, 28,28,1)) / 255

In [6]:
class DrawInput(Widget):
    
    def on_touch_down(self, touch):
        with self.canvas:
           if not self.collide_point(*touch.pos): return
           touch.ud["line"] = Line(points=(touch.x, touch.y), width=15)
        
    def on_touch_move(self, touch):
        if not self.collide_point(*touch.pos): return
        touch.ud["line"].points += (touch.x, touch.y)
  
# creating the App class
class MyApp(App):

    def build(self):
        Fl = FloatLayout()
  
        btnPredict = Button(text ='Predict',
                    size_hint =(.3, .2),
                    pos_hint ={'x': 0.7, 'y':0.8 })
        
        btnReset = Button(text = 'Reset', size_hint = (.3, .2), pos_hint ={'x': 0.7, 'y':0.6 })
        self.label = Label( text = '', size_hint = (.3, .6), pos_hint={'x':0.7, 'y':0.2})
        btnReset.bind( on_release=self.clear_canvas)
        btnPredict.bind (on_release=self.prediction)
        
        self.painter = DrawInput(size_hint = (.7, 1), pos_hint = {'x': 0.0, 'y': 0.0})
        
        
        # adding widget i.e button
        Fl.add_widget(btnPredict)
        Fl.add_widget(btnReset)
        Fl.add_widget(self.painter)
        Fl.add_widget(self.label)
        
        # return the layout
        return Fl
    def clear_canvas( self, obj):
        self.painter.canvas.clear()
        self.label.text = ""
    
    def prediction (self, event):
        kvyImg = self.painter.export_as_image()
        array = getImageNpArray( kvyImg)
        self.label.text = formatOutput( array)

In [7]:
MyApp().run()